Importing Libraries and Loading Models

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle

# cv = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open('./drive/MyDrive/Colab Notebooks/2 Sentiment Analysis (Basic)/3.1 BoW_Sentiment Model.pkl', "rb")))
cv = pickle.load(open('c1_BoW_Sentiment_Model.pkl', "rb"))


import joblib
classifier = joblib.load('/content/c2_Classifier_Sentiment_Model')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#function to convert input to list
def convert_to_sentences(paragraph):
    # Use regular expressions to split the paragraph into sentences based on full stops
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', paragraph)

    return sentences

Installing Anvil for hosting

In [ ]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=f685c8bdba5c081194715966833200cf5cb17b9caff8437569cceecd8e9e1a55
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("server_QLRCLOOEQNEP7OCMGPZCE6XF-4LE6YBLJOSLRNLJ7")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


Predicting sentiment using NaiveBayes Classifier

In [ ]:
@anvil.server.callable

def sentiment_calculator(sentence):

  #converting the input into a list
  sentence = convert_to_sentences(sentence)

  ps = PorterStemmer()

  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')

  corpus=[]
  size=len(sentence)

  for i in range(0, size):
    review = re.sub('[^a-zA-Z]', ' ', sentence[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

  x_pred= cv.transform(corpus).toarray()
  y_pred = classifier.predict(x_pred)

  # returning final results
  count_of_ones=0
  count_of_zeros=0

  for i in y_pred:
    if i==1:
      count_of_ones+=1
    else:
      count_of_zeros+=1

  pos_rating=str(float(count_of_ones)/len(y_pred))
  neg_rating=str(float(count_of_zeros)/len(y_pred))

  return pos_rating, neg_rating

Running on Anvil server

In [ ]:
anvil.server.wait_forever()